In [1]:
import tensorflow as tf

# Check if TensorFlow detects GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {gpus}")
else:
    print("No GPUs detected. TensorFlow is running on CPU.")

No GPUs detected. TensorFlow is running on CPU.


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # Enable all logs
import tensorflow as tf

# Check if TensorFlow detects GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {gpus}")
else:
    print("No GPUs detected. TensorFlow is running on CPU.")
    

No GPUs detected. TensorFlow is running on CPU.


In [ ]:
import tensorflow as tf

# Create a tensor and perform a computation
a = tf.random.normal([1000, 1000])
b = tf.random.normal([1000, 1000])
c = tf.matmul(a, b)

print("Computation complete. Check GPU usage in Task Manager or nvidia-smi.")

Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "c:\Python312\Lib\multiprocessing\util.py", line 221, in __call__
    if self._pid != getpid():
                    ^^^^^^^^
KeyboardInterrupt: 


In [4]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.backends.cudnn.version())  # Should return 8100 for cuDNN 8.1


True
90100


In [24]:
import torch
print(torch.version.cuda)  # Should print 11.2
print(torch.cuda.is_available())  # Should be True



11.8
True


In [25]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should return '11.2'
print(torch.backends.cudnn.version())  # Should return 8100 or similar


True
11.8
90100


In [ ]:
print(f"Predicted Index: {predicted_index}, Total Classes: {len(class_names)}")


ModuleNotFoundError: No module named 'jax'

In [6]:
pip show jax jaxlib


Name: jax
Version: 0.5.0
Summary: Differentiate, compile, and transform Numpy code.
Home-page: https://github.com/jax-ml/jax
Author: JAX team
Author-email: jax-dev@google.com
License: Apache-2.0
Location: C:\Users\parth\AppData\Roaming\Python\Python312\site-packages
Requires: jaxlib, ml_dtypes, numpy, opt_einsum, scipy
Required-by: 
---
Name: jaxlib
Version: 0.5.0
Summary: XLA library for JAX
Home-page: https://github.com/jax-ml/jax
Author: JAX team
Author-email: jax-dev@google.com
License: Apache-2.0
Location: C:\Users\parth\AppData\Roaming\Python\Python312\site-packages
Requires: ml-dtypes, numpy, scipy
Required-by: jax
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import json
import os
from tqdm import tqdm
import time
from collections import Counter

# Enable cuDNN benchmark
torch.backends.cudnn.benchmark = True

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Data configurations
DATA_DIR = 'C:\\Users\\parth\\Downloads\\Dataset1'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')
IMG_SIZE = 224  # Increased from 128 for better feature extraction
BATCH_SIZE = 64  # Increased batch size for faster training

# Enhanced data augmentations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
valid_dataset = datasets.ImageFolder(VALID_DIR, transform=test_transform)

# Handle class imbalance
class_counts = torch.tensor([len([x for x in os.listdir(os.path.join(TRAIN_DIR, cls)) 
                           for cls in train_dataset.classes])
class_weights = 1. / class_counts.float()
samples_weights = class_weights[train_dataset.targets]
sampler = WeightedRandomSampler(samples_weights, len(samples_weights))

# Create data loaders with optimized settings
num_workers = min(4, os.cpu_count())
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler,
                         num_workers=num_workers, pin_memory=True, persistent_workers=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=num_workers, pin_memory=True, persistent_workers=True)

# Use pretrained model with transfer learning
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

# Modified classifier head
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.3),
    nn.Linear(512, len(train_dataset.classes))
model = model.to(device)

# Mixed precision training
scaler = GradScaler() if device.type == 'cuda' else None

# Optimizer with weight decay
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3,
                                        steps_per_epoch=len(train_loader), epochs=20)

# Label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

def train_epoch(model, loader, optimizer, scheduler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training', leave=False)
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        with autocast(enabled=(device.type == 'cuda')):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'Loss': loss.item(), 'Acc': f"{100*correct/total:.2f}%"})
    
    return running_loss / len(loader.dataset), 100 * correct / total

def validate(model, loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation', leave=False)
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({'Acc': f"{100*correct/total:.2f}%"})
    
    return running_loss / len(loader.dataset), 100 * correct / total

def train_model():
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0.0
    
    for epoch in range(20):
        print(f"\nEpoch {epoch+1}/20")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler)
        val_loss, val_acc = validate(model, valid_loader)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved best model!")
    
    return history

# Execute training
history = train_model()

# Rest of your visualization and evaluation code...

Epoch 8/20 [Train]:  93%|█████████▎| 882/953 [01:38<00:07,  8.91it/s, loss=1.59, acc=44.8]


KeyboardInterrupt: 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import time
import os
from tqdm import tqdm
import sys
import json

# Force immediate output flushing
class FlushableStdout:
    def __init__(self, stream):
        self.stream = stream

    def write(self, data):
        self.stream.write(data)
        self.stream.flush()

    def flush(self):
        self.stream.flush()

sys.stdout = FlushableStdout(sys.stdout)  # Wrap sys.stdout for immediate flushing

# Enable cuDNN benchmarking for faster training
torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current GPU: {torch.cuda.get_device_name(0)}")

# Rest of your code remains unchanged...

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
data_dir = "C:\\Users\\parth\\Downloads\\Dataset1"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "valid")

# Verify dataset paths
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Training directory not found: {train_dir}")
if not os.path.exists(val_dir):
    raise FileNotFoundError(f"Validation directory not found: {val_dir}")

# Create datasets with progress indication
print("Loading datasets...")
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")
print(f"Number of classes: {len(train_dataset.classes)}")

# Create data loaders with reduced workers for stability
num_workers = 2 if os.name == 'nt' else 4  # Fewer workers for Windows
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
                         num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, 
                       num_workers=num_workers, pin_memory=True)

# Model setup with progress indication
print("\nInitializing model...")
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
model = model.to(device)
print(f"Model initialized on {device.type.upper()}")

# Mixed precision training setup
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Training progress tracking
progress = []

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10):
    start_time = time.time()
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        
        # Training phase with progress bar
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]', 
                        file=sys.stdout, dynamic_ncols=True)
        
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            # Mixed precision backward pass
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Update statistics
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            train_bar.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Acc': f"{correct/total*100:.2f}%"
            })
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        
        val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]', 
                      file=sys.stdout, dynamic_ncols=True)
        
        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
                
                val_bar.set_postfix({
                    'Loss': f"{loss.item():.4f}",
                    'Acc': f"{val_correct/val_total*100:.2f}%"
                })
        
        # Calculate epoch metrics
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total * 100
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total * 100
        
        progress.append({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })
        
        # Print epoch summary
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")
        print("-" * 50)
        
        scheduler.step()
    
    print(f"\nTraining completed in: {time.time() - start_time:.2f} seconds")
    return progress

# Start training with exception handling
try:
    print("\nStarting training...")
    training_progress = train_model(model, train_loader, val_loader, 
                                   nn.CrossEntropyLoss(), optimizer, scheduler, epochs=10)
except Exception as e:
    print(f"\nError during training: {str(e)}")
    raise

# Save model safely
print("\nSaving model...")
try:
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'class_to_idx': train_dataset.class_to_idx
    }, "optimized_model.pth")
    print("Model saved successfully")
except Exception as e:
    print(f"Error saving model: {str(e)}")

# Save training progress
try:
    with open("training_progress.json", "w") as f:
        json.dump(training_progress, f, indent=4)
    print("Training progress saved")
except Exception as e:
    print(f"Error saving training progress: {str(e)}")

print("\nAll operations completed successfully!")

Using device: cuda
CUDA version: 11.8
Current GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Loading datasets...
Training samples: 30427, Validation samples: 30416
Number of classes: 147

Initializing model...
Model initialized on CUDA

Starting training...


C:\Users\parth\AppData\Local\Temp\ipykernel_26344\488539737.py:81: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None


Epoch 1/10 [Train]:   0%|          | 0/951 [00:00<?, ?it/s]

C:\Users\parth\AppData\Local\Temp\ipykernel_26344\488539737.py:107: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/10 [Val]: 100%|##########| 951/951 [03:09<00:00,  5.01it/s, Loss=4.3776, Acc=72.12%]

Epoch 1 Summary:
Train Loss: 1.4890 | Train Acc: 57.61%
Val Loss: 0.8641 | Val Acc: 72.12%
Learning Rate: 1.00e-03
--------------------------------------------------
Epoch 2/10 [Val]: 100%|##########| 951/951 [02:53<00:00,  5.49it/s, Loss=3.3221, Acc=76.92%]

Epoch 2 Summary:
Train Loss: 0.8014 | Train Acc: 73.70%
Val Loss: 0.6740 | Val Acc: 76.92%
Learning Rate: 9.76e-04
--------------------------------------------------
Epoch 3/10 [Val]: 100%|##########| 951/951 [02:44<00:00,  5.78it/s, Loss=1.3781, Acc=79.49%]

Epoch 3 Summary:
Train Loss: 0.6375 | Train Acc: 78.31%
Val Loss: 0.5859 | Val Acc: 79.49%
Learning Rate: 9.05e-04
--------------------------------------------------
Epoch 4/10 [Val]: 100%|##########| 951/951 [02:42<00:00,  5.84it/s, Loss=1.9553, Acc=81.55%]

Epoch 4 Summary:
Train Loss: 0.5372 | Train Acc: 81.19%
Val Loss: 0.5338 | Val Acc: 81.55%
Learning Rate: 7.94e-04
-----------

In [ ]:
# ... (previous code remains the same until mixed precision setup)

# Mixed precision setup with version check
if torch.cuda.is_available():
    try:
        # For PyTorch 2.0+
        from torch.amp import autocast, GradScaler
        scaler = GradScaler()
        AUTOCAST_ARGS = {'device_type': 'cuda'}
        log_info("Using torch.amp for mixed precision")
    except ImportError:
        # For older versions
        from torch.cuda.amp import autocast, GradScaler
        scaler = GradScaler()
        AUTOCAST_ARGS = {}
        log_info("Using torch.cuda.amp for mixed precision")
else:
    autocast = None
    scaler = None
    AUTOCAST_ARGS = {}
    log_info("Mixed precision not available (CPU only)")

# ... (rest of the code remains same until training loop)

# In the training loop's forward pass:
with autocast(**AUTOCAST_ARGS) if autocast else nullcontext():
    outputs = model(images)
    loss = criterion(outputs, labels)

INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE


[2025-02-26 17:02:04] Using device: cuda
[2025-02-26 17:02:04] CUDA version: 11.8
[2025-02-26 17:02:04] Current GPU: NVIDIA GeForce RTX 3050 Laptop GPU
[2025-02-26 17:02:04] Using base directory: C:\Users\parth\Downloads\Dataset1
[2025-02-26 17:02:04] Verifying datasets...
[2025-02-26 17:02:18] Loaded train1 dataset from C:\Users\parth\Downloads\Dataset1\train1 with 3173 samples
[2025-02-26 17:02:19] Loaded train2 dataset from C:\Users\parth\Downloads\Dataset1\train2 with 28268 samples
[2025-02-26 17:02:19] Loaded train3 dataset from C:\Users\parth\Downloads\Dataset1\train3 with 609 samples
[2025-02-26 17:02:19] Loaded valid1 dataset from C:\Users\parth\Downloads\Dataset1\valid1 with 3173 samples
[2025-02-26 17:02:19] Loaded valid2 dataset from C:\Users\parth\Downloads\Dataset1\valid2 with 28268 samples
[2025-02-26 17:02:19] Loaded valid3 dataset from C:\Users\parth\Downloads\Dataset1\valid3 with 609 samples
[2025-02-26 17:02:19] Combined training samples: 32050
[2025-02-26 17:02:19] C

Epoch 1 [Train]:   0%|          | 0/2004 [00:00<?, ?it/s]

[2025-02-26 17:02:19] Error in training batch 0: autocast.__init__() missing 1 required positional argument: 'device_type'


Epoch 1 [Train]:   0%|          | 0/2004 [00:00<?, ?it/s]

[2025-02-26 17:02:19] Training failed: autocast.__init__() missing 1 required positional argument: 'device_type'


TypeError: autocast.__init__() missing 1 required positional argument: 'device_type'

In [2]:
import torch
print(torch.__version__)  # Should be 2.0.0 or higher
print(torch.cuda.is_available())  # Should be True

2.6.0+cu118
True


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
import json
import sys

# Define your dataset directories
data_dir = r"C:\Users\parth\Downloads\Dataset1"
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")

# Enhanced Dataset class with feature extraction
class EnhancedLeafDataset(Dataset):
    def __init__(self, data_dir, transform=None, phase='train'):
        self.data_dir = data_dir
        self.transform = transform
        self.phase = phase
       
        # Get class names and create mapping
        self.classes = os.listdir(data_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
       
        # Get all image paths and labels
        self.images = []
        self.labels = []
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.images.append(os.path.join(class_dir, img_name))
                        self.labels.append(self.class_to_idx[class_name])

    def extract_features(self, image):
        """Extract important features from the leaf image"""
        img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
       
        # 1. Color Features
        hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
        color_features = {
            'mean_h': hsv[:,:,0].mean(),
            'mean_s': hsv[:,:,1].mean(),
            'mean_v': hsv[:,:,2].mean(),
            'std_h': hsv[:,:,0].std(),
            'std_s': hsv[:,:,1].std(),
            'std_v': hsv[:,:,2].std()
        }
       
        # 2. Disease Spot Detection
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([30, 255, 255])
        yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
       
        lower_dark = np.array([0, 0, 0])
        upper_dark = np.array([180, 255, 50])
        dark_mask = cv2.inRange(hsv, lower_dark, upper_dark)
       
        disease_features = {
            'yellow_spot_ratio': np.sum(yellow_mask > 0) / yellow_mask.size,
            'dark_spot_ratio': np.sum(dark_mask > 0) / dark_mask.size
        }
       
        # 3. Texture Features
        gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
        glcm = np.zeros((8,8))
        normalized_gray = ((gray/16).astype(np.uint8)).clip(0, 7)
       
        for i in range(normalized_gray.shape[0]-1):
            for j in range(normalized_gray.shape[1]-1):
                i_val = normalized_gray[i,j]
                j_val = normalized_gray[i,j+1]
                glcm[i_val,j_val] += 1
       
        glcm = glcm / (glcm.sum() + 1e-6)  # Added epsilon to prevent division by zero
       
        texture_features = {
            'contrast': np.sum(np.square(np.arange(8) - np.arange(8)[:,np.newaxis]) * glcm),
            'homogeneity': np.sum(glcm / (1 + np.square(np.arange(8) - np.arange(8)[:,np.newaxis])))
        }
       
        # Combine all features
        features = {**color_features, **disease_features, **texture_features}
        return features
   
    def __len__(self):
        return len(self.images)
   
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
       
        # Load and convert image
        image = Image.open(img_path).convert('RGB')
       
        # Extract features
        features = self.extract_features(image)
       
        # Transform image for CNN
        if self.transform:
            image = self.transform(image)
       
        return {
            'image': image,
            'features': torch.tensor([list(features.values())], dtype=torch.float32).squeeze(),
            'label': label
        }

if __name__ == "__main__":
    model, progress = train_model(data_dir, num_epochs=10)


NameError: name 'train_model' is not defined

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.models as models
import time
import os
from tqdm import tqdm
import sys
import json
from PIL import Image
from contextlib import nullcontext

# Immediate verification output
print("INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE", flush=True)
time.sleep(2)

def log_info(message):
    """Timestamped logging with immediate flush"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}", flush=True)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_info(f"Using device: {device}")
if device.type == 'cuda':
    log_info(f"CUDA version: {torch.version.cuda}")
    log_info(f"Current GPU: {torch.cuda.get_device_name(0)}")

# Image conversion with error handling
def convert_to_rgb(img):
    """Convert any image to RGB format with error handling"""
    try:
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            return img.convert('RGBA').convert('RGB')  # Flatten transparency
        return img.convert('RGB')
    except Exception as e:
        log_info(f"Image conversion error: {str(e)}")
        return img  # Return original if conversion fails

# Data transforms with error handling
def safe_transform(transform):
    def wrapper(img):
        try:
            return transform(img)
        except Exception as e:
            log_info(f"Transform error: {str(e)}")
            return None
    return wrapper

# Data augmentation with error handling
train_transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset verification
def verify_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()  # Verify file integrity
                img.close()
            except Exception as e:
                log_info(f"Corrupted file: {os.path.join(root, file)} - {str(e)}")
                raise

# Base directory
base_dir = "C:\\Users\\parth\\Downloads\\Dataset1"
log_info(f"Using base directory: {base_dir}")

# Verify datasets
log_info("Verifying datasets...")
for subdir in ["train1", "train2", "train3", "valid1", "valid2", "valid3"]:
    path = os.path.join(base_dir, subdir)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Directory not found: {path}")
    verify_images(path)

# Load datasets with error handling
def load_dataset(directory, transform, subset_name):
    try:
        dataset = datasets.ImageFolder(root=directory, transform=safe_transform(transform))
        log_info(f"Loaded {subset_name} dataset from {directory} with {len(dataset)} samples")
        return dataset
    except Exception as e:
        log_info(f"Error loading {subset_name} dataset: {str(e)}")
        return None

# Load training datasets
train_datasets = []
for i in range(1, 4):
    dir_path = os.path.join(base_dir, f"train{i}")
    dataset = load_dataset(dir_path, train_transform, f"train{i}")
    if dataset:
        train_datasets.append(dataset)

# Load validation datasets
val_datasets = []
for i in range(1, 4):
    dir_path = os.path.join(base_dir, f"valid{i}")
    dataset = load_dataset(dir_path, val_transform, f"valid{i}")
    if dataset:
        val_datasets.append(dataset)

# Combine datasets
combined_train = ConcatDataset(train_datasets)
combined_val = ConcatDataset(val_datasets)
log_info(f"Combined training samples: {len(combined_train)}")
log_info(f"Combined validation samples: {len(combined_val)}")

# DataLoaders with debugging settings
num_workers = 0  # Set to 0 for debugging
train_loader = DataLoader(combined_train, batch_size=16, shuffle=True, 
                         num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(combined_val, batch_size=16, shuffle=False, 
                       num_workers=num_workers, pin_memory=True)

# Model setup
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_datasets[0].classes))
model = model.to(device)

# Mixed precision setup with version check
if torch.cuda.is_available():
    try:
        # For PyTorch 2.0+
        from torch.amp import autocast, GradScaler
        scaler = GradScaler()
        AUTOCAST_ARGS = {'device_type': 'cuda'}
        log_info("Using torch.amp for mixed precision")
    except ImportError:
        # For older versions
        from torch.cuda.amp import autocast, GradScaler
        scaler = GradScaler()
        AUTOCAST_ARGS = {}
        log_info("Using torch.cuda.amp for mixed precision")
else:
    autocast = nullcontext
    scaler = None
    AUTOCAST_ARGS = {}
    log_info("Mixed precision not available (CPU only)")

# Training parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

# Training loop with enhanced error handling
def train_model(epochs=15):
    best_acc = 0.0
    for epoch in range(epochs):
        log_info(f"Starting epoch {epoch+1}/{epochs}")
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        
        # Training phase
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]')
        for batch_idx, (inputs, labels) in enumerate(train_bar):
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with autocast(**AUTOCAST_ARGS) if torch.cuda.is_available() else nullcontext():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                if scaler:
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                train_bar.set_postfix({
                    'Loss': f"{loss.item():.4f}",
                    'Acc': f"{100*correct/total:.2f}%"
                })
                
                if batch_idx % 50 == 0:
                    log_info(f"Processed {batch_idx} training batches")
            
            except Exception as e:
                log_info(f"Error in training batch {batch_idx}: {str(e)}")
                raise
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1} [Val]')
            for batch_idx, (inputs, labels) in enumerate(val_bar):
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = outputs.max(1)
                    val_total += labels.size(0)
                    val_correct += predicted.eq(labels).sum().item()
                    
                    val_bar.set_postfix({
                        'Loss': f"{loss.item():.4f}",
                        'Acc': f"{100*val_correct/val_total:.2f}%"
                    })
                    
                    if batch_idx % 20 == 0:
                        log_info(f"Processed {batch_idx} validation batches")
                
                except Exception as e:
                    log_info(f"Error in validation batch {batch_idx}: {str(e)}")
                    raise
        
        # Calculate metrics
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100 * correct / total
        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = 100 * val_correct / val_total
        
        log_info(f"Epoch {epoch+1} Results:")
        log_info(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.2f}%")
        log_info(f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.2f}%")
        
        # Save best model
        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_epoch_acc
            }, "best_model.pth")
            log_info("Saved best model")
        
        scheduler.step()
    
    log_info("Training complete")

# Execute training
try:
    log_info("Starting training...")
    train_model(epochs=15)
except Exception as e:
    log_info(f"Training failed: {str(e)}")
    raise

INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE
[2025-02-26 17:08:54] Using device: cuda
[2025-02-26 17:08:54] CUDA version: 11.8
[2025-02-26 17:08:54] Current GPU: NVIDIA GeForce RTX 3050 Laptop GPU
[2025-02-26 17:08:54] Using base directory: C:\Users\parth\Downloads\Dataset1
[2025-02-26 17:08:54] Verifying datasets...
[2025-02-26 17:09:08] Loaded train1 dataset from C:\Users\parth\Downloads\Dataset1\train1 with 3173 samples
[2025-02-26 17:09:08] Loaded train2 dataset from C:\Users\parth\Downloads\Dataset1\train2 with 28268 samples
[2025-02-26 17:09:08] Loaded train3 dataset from C:\Users\parth\Downloads\Dataset1\train3 with 609 samples
[2025-02-26 17:09:08] Loaded valid1 dataset from C:\Users\parth\Downloads\Dataset1\valid1 with 3173 samples
[2025-02-26 17:09:08] Loaded valid2 dataset from C:\Users\parth\Downloads\Dataset1\valid2 with 28268 samples
[2025-02-26 17:09:08] Loaded valid3 dataset from C:\Users\parth\Downloads\Dataset1\valid3 with 609 samples
[2025-02-26 17:09:08] Combined tr

Epoch 1 [Train]:   0%|          | 0/2004 [00:01<?, ?it/s, Loss=3.7906, Acc=0.00%]

[2025-02-26 17:09:10] Processed 0 training batches


Epoch 1 [Train]:   2%|▏         | 50/2004 [00:12<08:32,  3.81it/s, Loss=2.7647, Acc=32.60%]

[2025-02-26 17:09:21] Processed 50 training batches


...

Epoch 15 [Val]: 100%|█████████▉| 1999/2004 [09:06<00:00, 11.40it/s, Loss=0.0312, Acc=99.39%]

[2025-02-26 21:47:25] Processed 2000 validation batches


Epoch 15 [Val]: 100%|██████████| 2004/2004 [09:06<00:00,  3.66it/s, Loss=0.0008, Acc=99.39%]

[2025-02-26 21:47:26] Epoch 15 Results:
[2025-02-26 21:47:26] Train Loss: 0.0911 | Train Acc: 97.15%
[2025-02-26 21:47:26] Val Loss: 0.0195 | Val Acc: 99.39%


[2025-02-26 21:47:26] Saved best model
[2025-02-26 21:47:26] Training complete


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.models as models
import time
import os
from tqdm import tqdm
import sys
import json
from PIL import Image
from contextlib import nullcontext

# Immediate verification output
print("INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE", flush=True)
time.sleep(2)

def log_info(message):
    """Timestamped logging with immediate flush"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}", flush=True)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_info(f"Using device: {device}")
if device.type == 'cuda':
    log_info(f"CUDA version: {torch.version.cuda}")
    log_info(f"Current GPU: {torch.cuda.get_device_name(0)}")

# Image conversion with error handling
def convert_to_rgb(img):
    """Convert any image to RGB format with error handling"""
    try:
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            return img.convert('RGBA').convert('RGB')  # Flatten transparency
        return img.convert('RGB')
    except Exception as e:
        log_info(f"Image conversion error: {str(e)}")
        return img  # Return original if conversion fails

# Data transforms with error handling
def safe_transform(transform):
    def wrapper(img):
        try:
            return transform(img)
        except Exception as e:
            log_info(f"Transform error: {str(e)}")
            return None
    return wrapper

# Data augmentation with error handling
train_transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset verification
def verify_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()  # Verify file integrity
                img.close()
            except Exception as e:
                log_info(f"Corrupted file: {os.path.join(root, file)} - {str(e)}")
                raise

# Base directory
base_dir = "C:\\Users\\parth\\Downloads\\Dataset1"
log_info(f"Using base directory: {base_dir}")

# Verify datasets
log_info("Verifying datasets...")
for subdir in ["train1", "train2", "train3", "valid1", "valid2", "valid3"]:
    path = os.path.join(base_dir, subdir)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Directory not found: {path}")
    verify_images(path)

# Load datasets with error handling
def load_dataset(directory, transform, subset_name):
    try:
        dataset = datasets.ImageFolder(root=directory, transform=safe_transform(transform))
        log_info(f"Loaded {subset_name} dataset from {directory} with {len(dataset)} samples")
        return dataset
    except Exception as e:
        log_info(f"Error loading {subset_name} dataset: {str(e)}")
        return None

# Load training datasets
train_datasets = []
for i in range(1, 4):
    dir_path = os.path.join(base_dir, f"train{i}")
    dataset = load_dataset(dir_path, train_transform, f"train{i}")
    if dataset:
        train_datasets.append(dataset)

# Load validation datasets
val_datasets = []
for i in range(1, 4):
    dir_path = os.path.join(base_dir, f"valid{i}")
    dataset = load_dataset(dir_path, val_transform, f"valid{i}")
    if dataset:
        val_datasets.append(dataset)

# Combine datasets
combined_train = ConcatDataset(train_datasets)
combined_val = ConcatDataset(val_datasets)
log_info(f"Combined training samples: {len(combined_train)}")
log_info(f"Combined validation samples: {len(combined_val)}")

# DataLoaders with debugging settings
num_workers = 0  # Set to 0 for debugging
train_loader = DataLoader(combined_train, batch_size=16, shuffle=True, 
                         num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(combined_val, batch_size=16, shuffle=False, 
                       num_workers=num_workers, pin_memory=True)

# Model setup
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_datasets[0].classes))
model = model.to(device)

# Mixed precision setup with version check
if torch.cuda.is_available():
    try:
        # For PyTorch 2.0+
        from torch.amp import autocast, GradScaler
        scaler = GradScaler()
        AUTOCAST_ARGS = {'device_type': 'cuda'}
        log_info("Using torch.amp for mixed precision")
    except ImportError:
        # For older versions
        from torch.cuda.amp import autocast, GradScaler
        scaler = GradScaler()
        AUTOCAST_ARGS = {}
        log_info("Using torch.cuda.amp for mixed precision")
else:
    autocast = nullcontext
    scaler = None
    AUTOCAST_ARGS = {}
    log_info("Mixed precision not available (CPU only)")

# Training parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

# Training loop with enhanced error handling
def train_model(epochs=15):
    best_acc = 0.0
    for epoch in range(epochs):
        log_info(f"Starting epoch {epoch+1}/{epochs}")
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        
        # Training phase
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]')
        for batch_idx, (inputs, labels) in enumerate(train_bar):
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with autocast(**AUTOCAST_ARGS) if torch.cuda.is_available() else nullcontext():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                if scaler:
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                train_bar.set_postfix({
                    'Loss': f"{loss.item():.4f}",
                    'Acc': f"{100*correct/total:.2f}%"
                })
                
                if batch_idx % 50 == 0:
                    log_info(f"Processed {batch_idx} training batches")
            
            except Exception as e:
                log_info(f"Error in training batch {batch_idx}: {str(e)}")
                raise
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1} [Val]')
            for batch_idx, (inputs, labels) in enumerate(val_bar):
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = outputs.max(1)
                    val_total += labels.size(0)
                    val_correct += predicted.eq(labels).sum().item()
                    
                    val_bar.set_postfix({
                        'Loss': f"{loss.item():.4f}",
                        'Acc': f"{100*val_correct/val_total:.2f}%"
                    })
                    
                    if batch_idx % 20 == 0:
                        log_info(f"Processed {batch_idx} validation batches")
                
                except Exception as e:
                    log_info(f"Error in validation batch {batch_idx}: {str(e)}")
                    raise
        
        # Calculate metrics
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100 * correct / total
        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = 100 * val_correct / val_total
        
        log_info(f"Epoch {epoch+1} Results:")
        log_info(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.2f}%")
        log_info(f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.2f}%")
        
        # Save best model
        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_epoch_acc
            }, "best_model.pth")
            log_info("Saved best model")
        
        scheduler.step()
    
    log_info("Training complete")

# Execute training
try:
    log_info("Starting training...")
    train_model(epochs=15)
except Exception as e:
    log_info(f"Training failed: {str(e)}")
    raise

INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE
[2025-02-28 21:16:58] Using device: cuda
[2025-02-28 21:16:58] CUDA version: 11.8
[2025-02-28 21:16:58] Current GPU: NVIDIA GeForce RTX 3050 Laptop GPU
[2025-02-28 21:16:58] Using base directory: C:\Users\parth\Downloads\Dataset1
[2025-02-28 21:16:58] Verifying datasets...
[2025-02-28 21:20:47] Loaded train1 dataset from C:\Users\parth\Downloads\Dataset1\train1 with 31441 samples
[2025-02-28 21:20:47] Error loading train2 dataset: Couldn't find any class folder in C:\Users\parth\Downloads\Dataset1\train2.
[2025-02-28 21:20:47] Loaded train3 dataset from C:\Users\parth\Downloads\Dataset1\train3 with 609 samples
[2025-02-28 21:20:48] Loaded valid1 dataset from C:\Users\parth\Downloads\Dataset1\valid1 with 31441 samples
[2025-02-28 21:20:48] Error loading valid2 dataset: Couldn't find any class folder in C:\Users\parth\Downloads\Dataset1\valid2.
[2025-02-28 21:20:48] Loaded valid3 dataset from C:\Users\parth\Downloads\Dataset1\valid3 with 609 s

Epoch 1 [Train]:   0%|          | 0/2004 [00:01<?, ?it/s, Loss=4.3002, Acc=0.00%]

[2025-02-28 21:20:51] Processed 0 training batches


Epoch 1 [Train]:   2%|▏         | 50/2004 [00:16<09:35,  3.40it/s, Loss=2.8043, Acc=28.43%]

[2025-02-28 21:21:07] Processed 50 training batches


...

[2025-02-28 21:23:38] Processed 550 training batches


Epoch 1 [Train]:  30%|██▉       | 600/2004 [03:02<06:17,  3.72it/s, Loss=0.9685, Acc=57.00%]

[2025-02-28 21:23:53] Processed 600 training batches


Epoch 1 [Train]:  31%|███       | 621/2004 [03:08<06:59,  3.30it/s, Loss=0.4377, Acc=57.56%]


KeyboardInterrupt: 

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import time
import os
from tqdm import tqdm
from PIL import Image
from contextlib import nullcontext

# Immediate verification output
print("INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE", flush=True)
time.sleep(2)

def log_info(message):
    """Timestamped logging with immediate flush"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}", flush=True)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_info(f"Using device: {device}")
if device.type == 'cuda':
    log_info(f"CUDA version: {torch.version.cuda}")
    log_info(f"Current GPU: {torch.cuda.get_device_name(0)}")

# Image conversion with error handling
def convert_to_rgb(img):
    try:
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            return img.convert('RGBA').convert('RGB')
        return img.convert('RGB')
    except Exception as e:
        log_info(f"Image conversion error: {str(e)}")
        return img

# Verify dataset directory
base_dir = "C:\\Users\\parth\\Downloads\\Dataset1"
log_info(f"Using base directory: {base_dir}")

def verify_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()  # Verify file integrity
                try:
                    img.getexif()
                except Exception as e:
                    log_info(f"Corrupt EXIF data in: {file_path} - {str(e)}")
                img.close()
            except Exception as e:
                log_info(f"Corrupted file: {file_path} - {str(e)}")
                raise

# Dataset verification
for subdir in ["train1", "valid1"]:
    path = os.path.join(base_dir, subdir)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Directory not found: {path}")
    verify_images(path)

# Data augmentation with error handling
train_transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=os.path.join(base_dir, "train1"), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(base_dir, "valid1"), transform=val_transform)

log_info(f"Loaded {len(train_dataset)} training samples")
log_info(f"Loaded {len(val_dataset)} validation samples")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

# Model setup
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
model = model.to(device)

# Mixed precision setup
try:
    from torch.cuda.amp import autocast, GradScaler
    scaler = GradScaler()
    AUTOCAST_ARGS = {}
    log_info("Using torch.cuda.amp for mixed precision")
except ImportError:
    autocast = nullcontext
    scaler = None
    AUTOCAST_ARGS = {}
    log_info("Mixed precision not available (CPU only)")

# Training parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

def train_model(epochs=15):
    best_acc = 0.0
    for epoch in range(epochs):
        log_info(f"Starting epoch {epoch+1}/{epochs}")
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]')
        for batch_idx, (inputs, labels) in enumerate(train_bar):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            train_bar.set_postfix({'Loss': f"{loss.item():.4f}", 'Acc': f"{100 * correct / max(1, total):.2f}%"})

        scheduler.step()
        
        val_correct, val_total = 0, 0
        model.eval()
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1} [Val]')
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_acc = 100 * val_correct / max(1, val_total)
        log_info(f"Epoch {epoch+1} - Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model_70_class.pth")
            log_info("Saved best model")

    log_info("Training complete")

try:
    log_info("Starting training...")
    train_model(epochs=15)
except Exception as e:
    log_info(f"Training failed: {str(e)}")
    raise


INITIAL OUTPUT TEST - VERIFY THIS IS VISIBLE


[2025-03-01 01:12:43] Using device: cuda
[2025-03-01 01:12:43] CUDA version: 11.8
[2025-03-01 01:12:43] Current GPU: NVIDIA GeForce RTX 3050 Laptop GPU
[2025-03-01 01:12:43] Using base directory: C:\Users\parth\Downloads\Dataset1
[2025-03-01 01:15:36] Loaded 31441 training samples
[2025-03-01 01:15:36] Loaded 31441 validation samples
[2025-03-01 01:15:37] Using torch.cuda.amp for mixed precision
[2025-03-01 01:15:37] Starting training...
[2025-03-01 01:15:37] Starting epoch 1/15


C:\Users\parth\AppData\Local\Temp\ipykernel_34720\662562834.py:104: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1 [Train]:   0%|          | 0/1966 [00:00<?, ?it/s]C:\Users\parth\AppData\Local\Temp\ipykernel_34720\662562834.py:130: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 [Val]: 100%|██████████| 1966/1966 [18:30<00:00,  1.77it/s] 

[2025-03-01 01:51:17] Epoch 1 - Val Acc: 85.43%


[2025-03-01 01:51:18] Saved best model
[2025-03-01 01:51:18] Starting epoch 2/15


...


Epoch 14 [Val]: 100%|██████████| 1966/1966 [19:48<00:00,  1.65it/s] 

[2025-03-01 14:17:52] Epoch 14 - Val Acc: 99.44%


[2025-03-01 14:17:53] Saved best model
[2025-03-01 14:17:53] Starting epoch 15/15


Epoch 15 [Val]: 100%|██████████| 1966/1966 [19:50<00:00,  1.65it/s] 

[2025-03-01 14:57:12] Epoch 15 - Val Acc: 99.44%
[2025-03-01 14:57:12] Training complete


NameError: name 'app' is not defined

In [37]:
import math

# Assuming you have train_loader and val_loader defined
num_train_samples = len(train_loader.dataset)
train_batch_size = train_loader.batch_size
num_train_batches = math.ceil(num_train_samples / train_batch_size)

num_val_samples = len(val_loader.dataset)
val_batch_size = val_loader.batch_size
num_val_batches = math.ceil(num_val_samples / val_batch_size)

print(f"Number of training batches: {num_train_batches}")
print(f"Number of validation batches: {num_val_batches}")

Number of training batches: 1966
Number of validation batches: 1966


In [38]:
print(f"Number of classes being trained: {len(train_dataset.classes)}")

Number of classes being trained: 70


In [36]:
import torch

# Load the model checkpoint
file_path = "best_model.pth"  # Update with your file path
checkpoint = torch.load(file_path, map_location="cpu")

# Extract model state dictionary
model_state = checkpoint.get("model_state_dict", checkpoint)

# Find number of classes from the final fully connected layer
num_classes = None
for key, value in model_state.items():
    if "fc.weight" in key:  # Fully connected layer in ResNet
        num_classes = value.shape[0]  # Output neurons = number of classes
        break

print(f"Number of classes: {num_classes}")
# Assuming you have train_datasets (list of ImageFolder datasets)
num_classes = len(train_datasets[0].classes)
print(f"Number of classes: {num_classes}")
print(f"Classes: {train_datasets[0].classes}")

Number of classes: 42
Number of classes: 70
Classes: ['American Bollworm on Cotton', 'Anthracnose on Cotton', 'Apple__Healthy', 'Apple__Rotten', 'Army worm', 'Bacterial Blight in cotton', 'Banana__Healthy', 'Banana__Rotten', 'Becterial Blight in Rice', 'Bellpepper__Healthy', 'Bellpepper__Rotten', 'Brownspot', 'Carrot__Healthy', 'Carrot__Rotten', 'Common_Rust', 'Cotton Aphid', 'Cucumber__Healthy', 'Cucumber__Rotten', 'Flag Smut', 'Grape__Healthy', 'Grape__Rotten', 'Gray_Leaf_Spot', 'Guava__Healthy', 'Guava__Rotten', 'Healthy Maize', 'Healthy Wheat', 'Healthy cotton', 'Jujube__Healthy', 'Jujube__Rotten', 'Leaf Curl', 'Leaf smut', 'Mango__Healthy', 'Mango__Rotten', 'Mosaic sugarcane', 'Orange__Healthy', 'Orange__Rotten', 'Pomegranate__Healthy', 'Pomegranate__Rotten', 'Potato__Healthy', 'Potato__Rotten', 'RedRot sugarcane', 'RedRust sugarcane', 'Rice Blast', 'Strawberry__Healthy', 'Strawberry__Rotten', 'Sugarcane Healthy', 'Tomato__Healthy', 'Tomato__Rotten', 'Tungro', 'Wheat Brown leaf ru

In [29]:
for dataset in train_datasets:
    log_info(f"Individual train dataset length: {len(dataset)}")
for dataset in val_datasets:
    log_info(f"Individual validation dataset length: {len(dataset)}")
    
log_info(f"Combined training samples: {len(combined_train)}")
log_info(f"Combined validation samples: {len(combined_val)}")    

[2025-03-01 00:36:59] Individual train dataset length: 31441
[2025-03-01 00:36:59] Individual train dataset length: 609
[2025-03-01 00:36:59] Individual validation dataset length: 31441
[2025-03-01 00:36:59] Individual validation dataset length: 609
[2025-03-01 00:36:59] Combined training samples: 32050
[2025-03-01 00:36:59] Combined validation samples: 32050


In [ ]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, ConcatDataset

# Define dataset paths
all_train_paths = {"train1": "Q:\\train1", "train2": "Q:\\train2", "train3": "Q:\\train3"}
all_valid_paths = {"valid1": "Q:\\valid1", "valid2": "Q:\\valid2", "valid3": "Q:\\valid3"}

# Check existing paths
existing_train_paths = {k: v for k, v in all_train_paths.items() if os.path.exists(v)}
existing_valid_paths = {k: v for k, v in all_valid_paths.items() if os.path.exists(v)}

# Already trained datasets
trained_datasets = {"train1", "valid1"}  # Modify this based on actual trained sets

# Filter only new paths
new_train_paths = [v for k, v in existing_train_paths.items() if k not in trained_datasets]
new_valid_paths = [v for k, v in existing_valid_paths.items() if k not in trained_datasets]

if not new_train_paths and not new_valid_paths:
    print("No new data to train. Exiting.")
    exit()

# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
def load_datasets(paths, transform):
    datasets_list = []
    for path in paths:
        dataset = datasets.ImageFolder(root=path, transform=transform)
        datasets_list.append(dataset)
    return ConcatDataset(datasets_list) if datasets_list else None

# Load only the new data
new_train_data = load_datasets(new_train_paths, train_transform)
new_valid_data = load_datasets(new_valid_paths, val_transform)

# DataLoaders
train_loader = DataLoader(new_train_data, batch_size=16, shuffle=True, num_workers=4) if new_train_data else None
valid_loader = DataLoader(new_valid_data, batch_size=16, shuffle=False, num_workers=4) if new_valid_data else None

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=None)

# Determine the number of classes dynamically
if new_train_data:
    num_classes = len(new_train_data.datasets[0].classes)
else:
    num_classes = 42  # Default value if no dataset is loaded

# Initialize the fc layer with the correct number of classes
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

# Load existing model if available
model_path = "best_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    # Load the state dict, but only after ensuring the fc layer is correctly sized.
    model.load_state_dict(checkpoint["model_state_dict"], strict=False)
    print("Loaded existing model for further training.")
else:
    print("Training a new model from scratch.")

model = model.to(device)

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# Training function
def train_model(epochs=5):
    if not train_loader:
        print("No new training data available.")
        return
    
    model.train()
    for epoch in range(epochs):
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
        
        acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, Acc={acc:.2f}%")

# Train the model
train_model(epochs=5)

# Save updated model
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, "updated_model.pth")

print("Training complete. Model saved as updated_model.pth")


No new data to train. Exiting.
Loaded existing model for further training.
No new training data available.
Training complete. Model saved as updated_model.pth


: 

In [30]:
import os
import torchvision.datasets as datasets
import torchvision.transforms as transforms

base_dir = "C:\\Users\\parth\\Downloads\\Dataset1"
train1_dir = os.path.join(base_dir, "train1")

transform = transforms.ToTensor()

try:
    train1_dataset = datasets.ImageFolder(root=train1_dir, transform=transform)
    print(f"Loaded train1 dataset with {len(train1_dataset)} samples")
    print(f"Classes in train1: {train1_dataset.classes}")
except Exception as e:
    print(f"Error loading train1: {e}")

Loaded train1 dataset with 31441 samples
Classes in train1: ['American Bollworm on Cotton', 'Anthracnose on Cotton', 'Apple__Healthy', 'Apple__Rotten', 'Army worm', 'Bacterial Blight in cotton', 'Banana__Healthy', 'Banana__Rotten', 'Becterial Blight in Rice', 'Bellpepper__Healthy', 'Bellpepper__Rotten', 'Brownspot', 'Carrot__Healthy', 'Carrot__Rotten', 'Common_Rust', 'Cotton Aphid', 'Cucumber__Healthy', 'Cucumber__Rotten', 'Flag Smut', 'Grape__Healthy', 'Grape__Rotten', 'Gray_Leaf_Spot', 'Guava__Healthy', 'Guava__Rotten', 'Healthy Maize', 'Healthy Wheat', 'Healthy cotton', 'Jujube__Healthy', 'Jujube__Rotten', 'Leaf Curl', 'Leaf smut', 'Mango__Healthy', 'Mango__Rotten', 'Mosaic sugarcane', 'Orange__Healthy', 'Orange__Rotten', 'Pomegranate__Healthy', 'Pomegranate__Rotten', 'Potato__Healthy', 'Potato__Rotten', 'RedRot sugarcane', 'RedRust sugarcane', 'Rice Blast', 'Strawberry__Healthy', 'Strawberry__Rotten', 'Sugarcane Healthy', 'Tomato__Healthy', 'Tomato__Rotten', 'Tungro', 'Wheat Brown 

In [1]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, ConcatDataset

# Define dataset paths
base_path = "C:\\Users\\parth\\Downloads\\Dataset1"
all_train_paths = {"train2": os.path.join(base_path, "train2"), "train3": os.path.join(base_path, "train3")}
all_valid_paths = {"valid2": os.path.join(base_path, "valid2"), "valid3": os.path.join(base_path, "valid3")}

# Check existing paths
existing_train_paths = {k: v for k, v in all_train_paths.items() if os.path.exists(v)}
existing_valid_paths = {k: v for k, v in all_valid_paths.items() if os.path.exists(v)}

# Filter only new paths
new_train_paths = list(existing_train_paths.values())
new_valid_paths = list(existing_valid_paths.values())

# Print paths for debugging
print("Existing train paths:", existing_train_paths)
print("Existing valid paths:", existing_valid_paths)
print("New train paths:", new_train_paths)
print("New valid paths:", new_valid_paths)

if not new_train_paths and not new_valid_paths:
    print("No new data to train. Exiting.")
    exit()

# ... (rest of your code)

Existing train paths: {'train2': 'C:\\Users\\parth\\Downloads\\Dataset1\\train2', 'train3': 'C:\\Users\\parth\\Downloads\\Dataset1\\train3'}
Existing valid paths: {'valid2': 'C:\\Users\\parth\\Downloads\\Dataset1\\valid2', 'valid3': 'C:\\Users\\parth\\Downloads\\Dataset1\\valid3'}
New train paths: ['C:\\Users\\parth\\Downloads\\Dataset1\\train2', 'C:\\Users\\parth\\Downloads\\Dataset1\\train3']
New valid paths: ['C:\\Users\\parth\\Downloads\\Dataset1\\valid2', 'C:\\Users\\parth\\Downloads\\Dataset1\\valid3']


In [2]:
import os
import torchvision.datasets as datasets
import torchvision.transforms as transforms

base_path = "C:\\Users\\parth\\Downloads\\Dataset1\\train2"  # Test with train2
transform = transforms.ToTensor()

try:
    dataset = datasets.ImageFolder(root=base_path, transform=transform)
    print("Dataset loaded successfully.")
    print("Number of classes:", len(dataset.classes))
    print("Classes:", dataset.classes)
    print("Number of samples:", len(dataset))

    # Check the first few samples
    for i in range(min(5, len(dataset))):
        print(f"Sample {i}: Label={dataset[i][1]}")

except Exception as e:
    print(f"Dataset failed to load: {e}")

Dataset loaded successfully.
Number of classes: 28
Classes: ['Apple__Healthy', 'Apple__Rotten', 'Banana__Healthy', 'Banana__Rotten', 'Bellpepper__Healthy', 'Bellpepper__Rotten', 'Carrot__Healthy', 'Carrot__Rotten', 'Cucumber__Healthy', 'Cucumber__Rotten', 'Grape__Healthy', 'Grape__Rotten', 'Guava__Healthy', 'Guava__Rotten', 'Jujube__Healthy', 'Jujube__Rotten', 'Mango__Healthy', 'Mango__Rotten', 'Orange__Healthy', 'Orange__Rotten', 'Pomegranate__Healthy', 'Pomegranate__Rotten', 'Potato__Healthy', 'Potato__Rotten', 'Strawberry__Healthy', 'Strawberry__Rotten', 'Tomato__Healthy', 'Tomato__Rotten']
Number of samples: 28268
Sample 0: Label=0
Sample 1: Label=0
Sample 2: Label=0
Sample 3: Label=0
Sample 4: Label=0


In [10]:
import os
import torchvision.datasets as datasets
import torchvision.transforms as transforms

directories = [
    "C:\\Users\\parth\\Downloads\\Dataset1\\train2",
    "C:\\Users\\parth\\Downloads\\Dataset1\\train3",
    "C:\\Users\\parth\\Downloads\\Dataset1\\valid2",
    "C:\\Users\\parth\\Downloads\\Dataset1\\valid3"
]

transform = transforms.ToTensor()

for directory in directories:
    print(f"Testing directory: {directory}")
    try:
        dataset = datasets.ImageFolder(root=directory, transform=transform)
        print("Dataset loaded successfully.")
        print("Number of classes:", len(dataset.classes))
        print("Classes:", dataset.classes)
        print("Number of samples:", len(dataset))
    except Exception as e:
        print(f"Dataset failed to load: {e}")

Testing directory: C:\Users\parth\Downloads\Dataset1\train2
Dataset failed to load: Couldn't find any class folder in C:\Users\parth\Downloads\Dataset1\train2.
Testing directory: C:\Users\parth\Downloads\Dataset1\train3
Dataset loaded successfully.
Number of classes: 5
Classes: ['Black spot', 'Melanose', 'canker', 'greening', 'healthy']
Number of samples: 609
Testing directory: C:\Users\parth\Downloads\Dataset1\valid2
Dataset failed to load: Couldn't find any class folder in C:\Users\parth\Downloads\Dataset1\valid2.
Testing directory: C:\Users\parth\Downloads\Dataset1\valid3
Dataset loaded successfully.
Number of classes: 5
Classes: ['Black spot', 'Melanose', 'canker', 'greening', 'healthy']
Number of samples: 609


In [5]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, ConcatDataset

# Define dataset paths
base_path = "C:\\Users\\parth\\Downloads\\Dataset1"
train_paths = [os.path.join(base_path, "train2")]
valid_paths = [os.path.join(base_path, "valid2")]

# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
def load_datasets(paths, transform):
    datasets_list = []
    for path in paths:
        dataset = datasets.ImageFolder(root=path, transform=transform)
        datasets_list.append(dataset)
    return ConcatDataset(datasets_list) if datasets_list else None

# Load data
train_data = load_datasets(train_paths, train_transform)
valid_data = load_datasets(valid_paths, val_transform)

# DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_data, batch_size=16, shuffle=False, num_workers=4)

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=None)
num_classes = len(train_data.datasets[0].classes)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

# Optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
def train_model(epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
        # Calculate accuracy and print after each epoch
        acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, Acc={acc:.2f}%")

# Train and save
train_model(epochs=5)
torch.save({'model_state_dict': model.state_dict()}, "fruit_veg_model.pth")

KeyboardInterrupt: 

In [4]:
import torch

# Load the checkpoint (adjust the path to your .pth file)
checkpoint = torch.load("optimized_model.pth", map_location=torch.device('cpu'))  # Load on CPU

# Print keys in the checkpoint to see what's stored
print("Keys in the checkpoint:", checkpoint.keys())

# Inspect the contents
if 'state_dict' in checkpoint:
    print("\nModel architecture (state_dict) keys:", checkpoint['state_dict'].keys())
if 'epoch' in checkpoint:
    print("\nEpoch:", checkpoint['epoch'])
if 'optimizer' in checkpoint:
    print("\nOptimizer state:", checkpoint['optimizer'])
if 'class_to_idx' in checkpoint:
    print("\nClass-to-index mapping:", checkpoint['class_to_idx'])

Keys in the checkpoint: dict_keys(['model_state_dict', 'optimizer_state_dict', 'class_to_idx'])

Class-to-index mapping: {'American Bollworm on Cotton': 0, 'Anthracnose on Cotton': 1, 'Apple__Healthy': 2, 'Apple__Rotten': 3, 'Apple___alternaria_leaf_spot': 4, 'Apple___black_rot': 5, 'Apple___brown_spot': 6, 'Apple___gray_spot': 7, 'Apple___healthy': 8, 'Apple___rust': 9, 'Apple___scab': 10, 'Army worm': 11, 'Banana__Healthy': 12, 'Banana__Rotten': 13, 'Becterial Blight in Rice': 14, 'Bell_pepper___bacterial_spot': 15, 'Bell_pepper___healthy': 16, 'Bellpepper__Healthy': 17, 'Bellpepper__Rotten': 18, 'Black spot': 19, 'Blueberry___healthy': 20, 'Brownspot': 21, 'Canker': 22, 'Carrot__Healthy': 23, 'Carrot__Rotten': 24, 'Cassava___bacterial_blight': 25, 'Cassava___brown_streak_disease': 26, 'Cassava___green_mottle': 27, 'Cassava___healthy': 28, 'Cassava___mosaic_disease': 29, 'Cherry___healthy': 30, 'Cherry___powdery_mildew': 31, 'Coffee___healthy': 32, 'Coffee___red_spider_mite': 33, 'Co

In [5]:
checkpoint = torch.load("optimized_model.pth")
print(checkpoint.keys())  # See available keys


dict_keys(['model_state_dict', 'optimizer_state_dict', 'class_to_idx'])


In [1]:
import torch

# Load the checkpoint
checkpoint_path = "/mnt/data/optimized_model.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Extract class-to-index mapping
class_to_idx = checkpoint.get('class_to_idx', {})
num_classes = len(class_to_idx)

# Print model details
{
    "Number of classes": num_classes,
    "Checkpoint keys": list(checkpoint.keys())
}


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/optimized_model.pth'

In [1]:
# [Previous imports and setup remain the same until train_epoch function]

def train_epoch(model, loader, optimizer, scheduler, scaler, criterion, accumulation_steps=2):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    optimizer.zero_grad()
    
    print("Starting training epoch...")
    print(f"Total batches: {len(loader)}")
    print(f"Batch size: {loader.batch_size}")
    print("Memory usage before training:")
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
        print(f"GPU memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")
    
    pbar = tqdm(loader, desc='Training', leave=True)  # Changed leave to True
    for i, (inputs, labels) in enumerate(pbar):
        try:
            if i == 0:
                print(f"First batch shapes - Inputs: {inputs.shape}, Labels: {labels.shape}")
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            with autocast(enabled=(device.type == 'cuda')):
                outputs = model(inputs)
                loss = criterion(outputs, labels) / accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (i + 1) % accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()
            
            running_loss += loss.item() * inputs.size(0) * accumulation_steps
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            if i % 10 == 0:  # Print every 10 batches
                print(f"\nBatch {i}/{len(loader)}")
                print(f"Current loss: {loss.item():.4f}")
                print(f"Current accuracy: {100*correct/total:.2f}%")
                if torch.cuda.is_available():
                    print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
            
            pbar.set_postfix({
                'Loss': f"{loss.item() * accumulation_steps:.4f}",
                'Acc': f"{100*correct/total:.2f}%",
                'Batch': f"{i}/{len(loader)}"
            })
            
        except Exception as e:
            print(f"Error in batch {i}:")
            print(str(e))
            raise e
    
    return running_loss / len(loader.dataset), 100 * correct / total

# Main training loop with additional logging
def train_model(model, train_loader, valid_loader, num_epochs):
    print("Initializing training...")
    print(f"Total epochs: {num_epochs}")
    print(f"Training samples: {len(train_loader.dataset)}")
    print(f"Validation samples: {len(valid_loader.dataset)}")
    
    scaler = GradScaler(enabled=(device.type == 'cuda'))  # Updated GradScaler initialization
    
    # Optimizer with weight decay
    optimizer = optim.AdamW([
        {'params': [p for n, p in model.named_parameters() if 'classifier' not in n], 'lr': 1e-4},
        {'params': [p for n, p in model.named_parameters() if 'classifier' in n], 'lr': 1e-3}
    ], weight_decay=1e-4)
    
    # One Cycle learning rate scheduler
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=[1e-4, 1e-3],
        steps_per_epoch=len(train_loader),
        epochs=num_epochs,
        pct_start=0.3
    )
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0.0
    patience = 5
    patience_counter = 0
    
    try:
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            print("Memory status before epoch:")
            if torch.cuda.is_available():
                print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
                print(f"GPU memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")
            
            train_loss, train_acc = train_epoch(model, train_loader, optimizer, 
                                              scheduler, scaler, criterion)
            val_loss, val_acc = validate(model, valid_loader, criterion)
            
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            
            print(f"\nEpoch {epoch+1} Results:")
            print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
            print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
            
            if val_acc > best_acc:
                best_acc = val_acc
                print(f"New best accuracy: {best_acc:.2f}%")
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_acc': best_acc,
                }, 'best_model.pth')
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered!")
                    break
            
            # Clear cache after each epoch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    
    except Exception as e:
        print("Error during training:")
        print(str(e))
        raise e
    
    return history

# Execute training with error handling
if __name__ == '__main__':
    try:
        print("Creating data loaders...")
        train_loader, valid_loader = create_data_loaders(train_dataset, valid_dataset, BATCH_SIZE)
        
        print("Creating model...")
        model = create_model(len(train_dataset.classes))
        
        print("Starting training...")
        history = train_model(model, train_loader, valid_loader, NUM_EPOCHS)
        
        print("Creating training history plot...")
        plot_training_history(history)
        
    except Exception as e:
        print("Error in main execution:")
        print(str(e))
        raise e

Creating data loaders...
Error in main execution:
name 'create_data_loaders' is not defined


NameError: name 'create_data_loaders' is not defined

In [2]:
import torch
class_weights = torch.tensor([1.0 / count for count in class_counts], device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

NameError: name 'class_counts' is not defined

In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model state
checkpoint = torch.load("optimized_model.pth", map_location=device)

# Recreate the model architecture
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(checkpoint['class_to_idx']))  # Ensure the correct number of output classes

# Load model weights
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

# Define transformations (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load test dataset
data_dir = "C:\\Users\\parth\\Downloads\\Dataset1"
test_dir = os.path.join(data_dir, "valid")  # Assuming validation set for testing
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Get class names
class_names = test_dataset.classes

# Initialize lists for true and predicted labels
y_true = []
y_pred = []

# Perform inference
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


NameError: name 'os' is not defined

In [3]:
from collections import Counter

# Assuming train_dataset is your training dataset
class_counts = Counter(train_dataset.targets)  # Count samples per class
print("Class counts:", class_counts)

NameError: name 'train_dataset' is not defined

In [5]:
from collections import Counter

# Count samples per class
class_counts = {train_dataset.classes[idx]: count for idx, count in Counter(train_dataset.targets).items()}

# Print class distribution
for class_name, count in class_counts.items():
    print(f"{class_name}: {count} samples")


NameError: name 'train_dataset' is not defined

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Generate classification report
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    
    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    return report, cm

In [8]:
import torch

# Load the saved file
checkpoint = torch.load("trained_plant_disease_model.pth", map_location="cpu")

# Check the keys in the file
print(checkpoint.keys())

odict_keys(['conv1_1.weight', 'conv1_1.bias', 'conv1_2.weight', 'conv1_2.bias', 'conv2_1.weight', 'conv2_1.bias', 'conv2_2.weight', 'conv2_2.bias', 'conv3_1.weight', 'conv3_1.bias', 'conv3_2.weight', 'conv3_2.bias', 'conv4_1.weight', 'conv4_1.bias', 'conv4_2.weight', 'conv4_2.bias', 'conv5_1.weight', 'conv5_1.bias', 'conv5_2.weight', 'conv5_2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [5]:
# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Allow TensorFlow to use GPU memory efficiently
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Prevents memory overflow
        print("GPU is enabled and memory growth is set!")
    except RuntimeError as e:
        print(e)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Using GPU: ", tf.test.is_gpu_available())  # Deprecated, use the next line instead
print("GPU Device Name: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Using GPU:  False
GPU Device Name:  []


In [9]:
# Initialize the model
num_classes = 147  # Update this to match your dataset
model = PlantDiseaseModel(num_classes)

# Load the state dictionary
model.load_state_dict(torch.load("trained_plant_disease_model.pth", map_location="cpu"))

# Set the model to evaluation mode
model.eval()

NameError: name 'PlantDiseaseModel' is not defined

In [1]:
import tensorflow as tf

# Check available GPUs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("GPU Device Name: ", tf.config.list_physical_devices('GPU'))

# Check if TensorFlow is running on GPU
print("Is TensorFlow using GPU?", tf.test.is_built_with_cuda())

# Force TensorFlow to use GPU (if available)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Prevents memory overflow
        print("GPU is enabled and memory growth is set!")
    except RuntimeError as e:
        print(e)


Num GPUs Available:  0
GPU Device Name:  []
Is TensorFlow using GPU? False


In [2]:
import torch
print(torch.backends.cudnn.version())  # Should return a version like 9701
print(torch.cuda.is_available())  # Should return True
print(torch.backends.cudnn.enabled)  # Should return True


90100
True
True


In [3]:
90100
True
True

True

In [4]:
import torch
import time

def test_gpu_performance():
    # Check if CUDA is available
    if not torch.cuda.is_available():
        print("CUDA is not available. Running on CPU.")
        device = torch.device("cpu")
    else:
        print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
        device = torch.device("cuda")
    
    # Matrix size (larger = more computation)
    n = 5000
    
    # Create random matrices
    print(f"Creating {n}x{n} matrices...")
    a = torch.randn(n, n, device=device)
    b = torch.randn(n, n, device=device)
    
    # Warm-up run
    torch.matmul(a, b)
    torch.cuda.synchronize() if device.type == "cuda" else None
    
    # Measure performance
    print("Testing matrix multiplication performance...")
    start_time = time.time()
    
    # Perform matrix multiplication
    c = torch.matmul(a, b)
    
    # Make sure GPU operation is complete before timing
    torch.cuda.synchronize() if device.type == "cuda" else None
    
    elapsed_time = time.time() - start_time
    print(f"Matrix multiplication completed in: {elapsed_time:.4f} seconds")
    
    # Optional: calculate FLOPS (floating point operations per second)
    flops = 2 * n**3  # approximate flops for matrix multiplication
    gflops = flops / (elapsed_time * 1e9)
    print(f"Performance: {gflops:.2f} GFLOPS")
    
    return elapsed_time, gflops

if __name__ == "__main__":
    print("Starting GPU performance test...")
    test_gpu_performance()
    print("Test completed.")

Starting GPU performance test...
CUDA is available. Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Creating 5000x5000 matrices...
Testing matrix multiplication performance...
Matrix multiplication completed in: 0.1104 seconds
Performance: 2263.86 GFLOPS
Test completed.
